__I'm going to be working with a dataset regarding DAW usage.__

- A DAW (Digital Audio Workstation) is a software used for recording, editing, mixing, and otherwise producing
music or other audio (foley, sfx, voiceover, etc...).

- I couldn't find many datasets on this topic, and of what I could find, almost all came from one company that
locked them behind a paywall. The exception is this dataset out of Universitá Degli Studi di Milano:
Ludovico, Luca Andrea; Avanzini, Federico; Presti, Giorgio, 2025, "Data for: DAW Usage Survey (2023)",
https://doi.org/10.13130/RD_UNIMI/YNQHKI, UNIMI Dataverse, V2 

- For more information on data collection methods, the associated readme has been included in this notebook.

The file is a json rather than a csv, so the loading process is a bit more awkward

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
raw_daw_df = pd.DataFrame()
with open('daw_survey.json') as file:
    json_data = json.load(file)
    raw_daw_df = pd.json_normalize(json_data)
raw_daw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UseCases                 615 non-null    object 
 1   PersonalData.Gender      614 non-null    object 
 2   PersonalData.Age         614 non-null    object 
 3   PersonalData.Country     614 non-null    object 
 4   PersonalData.Instrument  614 non-null    object 
 5   PersonalData.Level       614 non-null    object 
 6   PersonalData.Company     614 non-null    object 
 7   PersonalData.Training    614 non-null    object 
 8   PersonalData.DAW         614 non-null    object 
 9   PersonalData.Source      614 non-null    object 
 10  PersonalData             0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 53.0+ KB


In [3]:
raw_daw_df.head()

,UseCases,PersonalData.Gender,PersonalData.Age,PersonalData.Country,PersonalData.Instrument,PersonalData.Level,PersonalData.Company,PersonalData.Training,PersonalData.DAW,PersonalData.Source,PersonalData
0,"[{'id': 0, 'OS': 'Windows', 'DAW': 'FL studio'...",Male,25-29,Italy,[Electronic self-taught],"[Hobby, Job]",No,Yes,FL studio,tgum,NaN
1,"[{'id': 0, 'OS': 'Windows', 'DAW': 'FL studio'...",Male,18-24,Italy,"[(electro)Acoustic with teacher, (electro)Acou...","[Hobby, Job]",No,No,FL studio,tgim,NaN
2,"[{'id': 0, 'OS': 'Windows', 'DAW': 'FL studio'...",Male,18-24,Italy,[(electro)Acoustic with teacher],[Hobby],No,No,FL studio,tgum,NaN
3,"[{'id': 0, 'OS': 'Mac', 'DAW': 'Nuendo', 'Star...",Male,30-34,Netherlands,"[(electro)Acoustic self-taught, Electronic sel...",[Job],Owner,Yes,Nuendo,rtaue,NaN
4,"[{'id': 0, 'OS': 'Windows', 'DAW': 'Reaper', '...",Male,25-29,Israel,"[(electro)Acoustic with teacher, (electro)Acou...","[Hobby, Job]",Employee,Yes,Reaper,rtaue,NaN


__Looks like the UseCases section is still nested, let's fix that.__

In [4]:
print(type(raw_daw_df["UseCases"][0]), "of", type(raw_daw_df["UseCases"][0][0]))

<class 'list'> of <class 'dict'>


We see that the use cases object is a list split up into a dictionary with each dict containing info for each DAW the user entered.
Below we can see that some of the elements in the dict are themselves lists, but that each one has the same set of keys.

In [5]:
use_cases_0 = raw_daw_df["UseCases"][0]
for x in use_cases_0:
    print(x,"\n")

{'id': 0, 'OS': 'Windows', 'DAW': 'FL studio', 'Start': '2012', 'End': '2023', 'Task': ['Draft', 'Tracking', 'Editing', 'Sequencing and Programming', 'Mixing', 'Mastering', 'Post-production', 'Scientific research or measurements'], 'Context': ['Music (Electronic)', 'Sampling', 'Sound design', 'Soundtrack', 'Speech recording'], 'Level': 'Semi-professional'} 

{'id': 1, 'OS': 'Windows', 'DAW': 'Audacity', 'Start': '2012', 'End': '2023', 'Task': ['Editing', 'Post-production', 'Scientific research or measurements'], 'Context': ['Music (Electronic)', 'Sampling', 'Speech recording'], 'Level': 'Semi-professional'} 

{'id': 2, 'OS': 'Windows', 'DAW': 'Tracktor', 'Start': '2012', 'End': '2016', 'Task': ['Live performance'], 'Context': ['Music (Electronic)', 'Sampling'], 'Level': 'Semi-professional'} 

{'id': 3, 'OS': 'Windows', 'DAW': 'Pro tools', 'Start': '2018', 'End': '2019', 'Task': ['Editing', 'Mixing'], 'Context': ['Music (Electronic)', 'Speech recording'], 'Level': 'Semi-professional'} 


__What's the best way to handle this? We could:__ 

    - Turn each of these dicts into a dataframe and add each to the id row it came from under a new column. This would require creating a column for every DAW represented,
    because having a column for UseCase.id would give a mix of DAWs for each column, which is less useful than having the DAWs together. This feels slightly redundant given
    that we have a DAW column under PersonalData, but that is restricted to one DAW per entrant, and doesn't include the depth of UseCases. The readme states that the 
    "PersonalData.DAW" data is "primary DAW they use," which could have slightly different interpretations between users.
    
    - Completely unpack these lists into individual columns. This would provide the most control in theory, but would be a nightmare to work with. Bad.
    
    - Rather than merging this data into the PersonalData list, we could handle it completely seperately. This lets us clearly see what the common uses are for each DAW
    in question, but loses out on the information connecting users to their DAWs, i.e. what age are/were most users of a given DAW? The UseCase dict contains start and end
    dates for their use, but does NOT include the age of the user. Other connecting data exists and would be lost, such as connections between instrument and DAW, level of
    training and DAW, etc...

I think the right combination of options 1 and 3 would be to create a DataFrame with columns for each DAW containing their corresponding dicts (discarding unneeded values 
such as the name of the DAW itself (already in the header) and ID) and then renaming the "PersonalData.DAW" column to "Primary DAW". The DataFrame can be either combined
with the original, or kept seperate as an associated DataFrame, where each row corresponds to the same user as the row with the same index in the original DataFrame.

The first step of this process is to initialize a DataFrame where all DAWs are marked as Unused

In [6]:
# First create a template for a blank row, representing the DAWs used by one user.
# Do this by first getting the names of all DAWs represented by iterating through
# the raw DataFrame
daw_names = set()
for i in raw_daw_df["UseCases"]:
    for j in i:
        daw_names.add(j["DAW"])

use_cases_by_daw_df = pd.DataFrame(columns=list(daw_names), index=np.arange(len(raw_daw_df)), data=None) 

use_cases_by_daw_df.head()

,,Adobe audition,Ardour,Nuendo,Reason,Waveform,Pro tools,Non daw studio,Acid pro,Soundbooth,...,Other,n-track studio,Mulab,Audiotool,Audacity,Digital performer,Tracktor,Renoise,Garageband,Biab
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now let's populate this new DataFrame with UseCases

In [7]:
# Once again iterate through all rows' UseCases column
current_row = 0
for i in raw_daw_df["UseCases"]:

    # Iterate through all items in the list
    for j in i:

        # Get the DAW we're looking at
        current_daw = j["DAW"]

        # Strip id and DAW from each dict, as id is not useful and
        # DAW is redundant since it is already the column label
        # Do this on a duplicate so as not to modify the raw data
        use_case = j
        use_case.pop('id')
        use_case.pop('DAW')

        # Add the element to the new DataFrame
        use_cases_by_daw_df.at[current_row, current_daw] = use_case

    current_row = current_row + 1

use_cases_by_daw_df.head()

,,Adobe audition,Ardour,Nuendo,Reason,Waveform,Pro tools,Non daw studio,Acid pro,Soundbooth,...,Other,n-track studio,Mulab,Audiotool,Audacity,Digital performer,Tracktor,Renoise,Garageband,Biab
0,NaN,NaN,NaN,NaN,NaN,NaN,"{'OS': 'Windows', 'Start': '2018', 'End': '201...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'OS': 'Windows', 'Start': '2012', 'End': '202...",NaN,"{'OS': 'Windows', 'Start': '2012', 'End': '201...",NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"{'OS': 'Mac', 'Start': '2014', 'End': '2023', ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Checking out the info for this shows us the number of uses reported for each DAW.

In [8]:
use_cases_by_daw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 615 entries, 0 to 614
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0                       14 non-null     object
 1   Adobe audition      29 non-null     object
 2   Ardour              18 non-null     object
 3   Nuendo              10 non-null     object
 4   Reason              20 non-null     object
 5   Waveform            2 non-null      object
 6   Pro tools           122 non-null    object
 7   Non daw studio      2 non-null      object
 8   Acid pro            7 non-null      object
 9   Soundbooth          1 non-null      object
 10  Ableton Live        226 non-null    object
 11  WaveLab             11 non-null     object
 12  Reaper              157 non-null    object
 13  SpectraLayers       1 non-null      object
 14  Soundforge          8 non-null      object
 15  Samplitude          4 non-null      object
 16  Music maker         1 non-null 

There appears to be an unlabeled column 0 with 14 non-null entries, let's check that out:

In [9]:
for x in use_cases_by_daw_df[""]:
    if str(x) != "nan":
        print(x)

{'OS': 'Windows', 'Start': '2012', 'End': '2023', 'Task': ['Draft', 'Tracking', 'Editing', 'Sequencing and Programming', 'Mixing', 'Mastering', 'Live sound engineering'], 'Context': ['Music (Electroacoustic)', 'Music (Electronic)'], 'Level': 'Amateur'}
{'OS': 'Windows', 'Start': '2011', 'End': '2023', 'Task': ['Draft', 'Tracking', 'Editing', 'Mixing', 'Post-production', 'Restoration'], 'Context': ['Music (Acoustic)', 'Music (Electroacoustic)', 'Sound design', 'Foley', 'Soundtrack', 'Speech recording'], 'Level': 'Professional'}
{'OS': 'Mac', 'Start': '2011', 'End': '2023', 'Task': ['Tracking', 'Editing', 'Mixing', 'Post-production', 'Restoration', 'Other'], 'Context': ['Music (Electronic)', 'Sampling', 'Sound art', 'Sound design', 'Foley', 'Soundtrack', 'Speech recording', 'Other'], 'Level': 'Semi-professional'}
{'OS': '', 'Start': '2010', 'End': '2023', 'Task': [], 'Context': [], 'Level': 'Professional'}
{'OS': 'Mac', 'Start': '2021', 'End': '2023', 'Task': ['Draft', 'Mixing', 'Masteri

It appears that users put the use cases they use DAWs for without specifying which DAW. For now, I'll simply label this column "Unspecified"

In [10]:
use_cases_by_daw_df.rename(columns={"": "Unspecified"}, inplace=True)
use_cases_by_daw_df.head()

,Unspecified,Adobe audition,Ardour,Nuendo,Reason,Waveform,Pro tools,Non daw studio,Acid pro,Soundbooth,...,Other,n-track studio,Mulab,Audiotool,Audacity,Digital performer,Tracktor,Renoise,Garageband,Biab
0,NaN,NaN,NaN,NaN,NaN,NaN,"{'OS': 'Windows', 'Start': '2018', 'End': '201...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'OS': 'Windows', 'Start': '2012', 'End': '202...",NaN,"{'OS': 'Windows', 'Start': '2012', 'End': '201...",NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"{'OS': 'Mac', 'Start': '2014', 'End': '2023', ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


While working on column headers, let's clean up both these use case ones and the PersonalData ones

In [11]:
use_cases_by_daw_df.columns.str.strip().str.title()

Index(['Unspecified', 'Adobe Audition', 'Ardour', 'Nuendo', 'Reason',
       'Waveform', 'Pro Tools', 'Non Daw Studio', 'Acid Pro', 'Soundbooth',
       'Ableton Live', 'Wavelab', 'Reaper', 'Spectralayers', 'Soundforge',
       'Samplitude', 'Music Maker', 'Soundtrap', 'Pyramix', 'Goldwave',
       'Bitwig Studio', 'Cakewalk', 'Mainstage', 'Sonic Visualizer',
       'Studio One', 'Logic', 'Serato Studio', 'Mixbus', 'Rx',
       'Acoustica Mixcraft', 'Cubase', 'Fl Studio', 'Other', 'N-Track Studio',
       'Mulab', 'Audiotool', 'Audacity', 'Digital Performer', 'Tracktor',
       'Renoise', 'Garageband', 'Biab'],
      dtype='object')

In [12]:
# Removing the UseCases column as we've handled that now
personal_data_df = raw_daw_df.drop(columns=['UseCases'])

Before cleaning the PersonalData headers, there's a suspicious column only labeled "PersonalData" let's get some more info.

In [13]:
personal_data_df['PersonalData'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 615 entries, 0 to 614
Series name: PersonalData
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 4.9 KB


Looks like it has no non-null values, so let's simply drop it and then clean the names

In [14]:
# Remove unneeded column
personal_data_df.drop(columns='PersonalData', inplace=True)

# Remove the PersonalData prefix from column names
new_names = dict()
for s in personal_data_df.columns:
    new_names[s] = s.strip().split(".")[1]

personal_data_df.rename(columns=new_names, inplace=True)

personal_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Gender      614 non-null    object
 1   Age         614 non-null    object
 2   Country     614 non-null    object
 3   Instrument  614 non-null    object
 4   Level       614 non-null    object
 5   Company     614 non-null    object
 6   Training    614 non-null    object
 7   DAW         614 non-null    object
 8   Source      614 non-null    object
dtypes: object(9)
memory usage: 43.4+ KB


__So after this phase of data cleaning, here are our current DataFrames:__

In [15]:
personal_data_df.head(10)

,Gender,Age,Country,Instrument,Level,Company,Training,DAW,Source
0,Male,25-29,Italy,[Electronic self-taught],"[Hobby, Job]",No,Yes,FL studio,tgum
1,Male,18-24,Italy,"[(electro)Acoustic with teacher, (electro)Acou...","[Hobby, Job]",No,No,FL studio,tgim
2,Male,18-24,Italy,[(electro)Acoustic with teacher],[Hobby],No,No,FL studio,tgum
3,Male,30-34,Netherlands,"[(electro)Acoustic self-taught, Electronic sel...",[Job],Owner,Yes,Nuendo,rtaue
4,Male,25-29,Israel,"[(electro)Acoustic with teacher, (electro)Acou...","[Hobby, Job]",Employee,Yes,Reaper,rtaue
5,Male,35-39,Latvia,[Electronic self-taught],[Hobby],No,No,Reaper,rtaue
6,Male,18-24,Italy,[],[Hobby],No,No,FL studio,tgim
7,Male,40-49,Spain,"[(electro)Acoustic with teacher, (electro)Acou...",[Job],Owner,Yes,Nuendo,rtaue
8,Male,40-49,United Kingdom,"[(electro)Acoustic self-taught, Electronic sel...",[Hobby],No,No,Cubase,rtaue
9,Male,18-24,Italy,"[(electro)Acoustic with teacher, (electro)Acou...","[Hobby, Job]",No,No,FL studio,tgum_s


In [16]:
use_cases_by_daw_df.head(10)

,Unspecified,Adobe audition,Ardour,Nuendo,Reason,Waveform,Pro tools,Non daw studio,Acid pro,Soundbooth,...,Other,n-track studio,Mulab,Audiotool,Audacity,Digital performer,Tracktor,Renoise,Garageband,Biab
0,NaN,NaN,NaN,NaN,NaN,NaN,"{'OS': 'Windows', 'Start': '2018', 'End': '201...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'OS': 'Windows', 'Start': '2012', 'End': '202...",NaN,"{'OS': 'Windows', 'Start': '2012', 'End': '201...",NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"{'OS': 'Mac', 'Start': '2014', 'End': '2023', ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,"{'OS': 'Mac', 'Start': '2013', 'End': '2023', ...","{'OS': 'Mac', 'Start': '2005', 'End': '2023', ...",NaN,"{'OS': 'Mac', 'Start': '2004', 'End': '2023', ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"{'OS': 'Windows', 'Start': '2012', 'End': '202...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


__MORE DATA:__
    
    - As requested, I'll import some data on search popularity for various DAWs represented in this survey. We previously
    identified the usage popularity of each DAW, so I'll narrow the focus of the Google Trends results to look at only
    the DAWs with over 50 users, those being, in order of popularity:
        Ableton Live, Reaper, Pro Tools, Logic, Cubase, FL Studio, and Audacity
    
    - Search terms won't necessarily represent DAW interest for Reaper, Logic, Audacity, and possibly even Pro Tools, 
    whose names all have alternate meanings. However, Google Trends has a feature where I can specify that I mean
    the audio editing software, so this shouldn't be a huge issue.

    - What is an issue though, is that Google Trends limits me to 5 items in a comparison. Reaper is a comparatively
    new DAW, so results going back several years won't be as useful. I'll also exclude Audacity, as I expect that we'll 
    see its use cases are different from these other DAWs. If I see later that doesn't end up being the case I may have
    to reconsider its exclusion, but the software is just fundamentally different from the other big names, so I feel
    comfortable doing so for now.

In [17]:
search_trends_global_df = pd.read_csv('multiTimeline.csv')
search_trends_global_df.head()

,,,,,Category: Music & Audio
Month,Ableton Live: (Worldwide),Cubase: (Worldwide),FL Studio: (Worldwide),Pro Tools: (Worldwide),Logic Pro: (Worldwide)
2004-01,12,96,100,77,11
2004-02,14,88,87,79,9
2004-03,14,80,91,69,9
2004-04,12,80,86,72,8


In [18]:
search_trends_by_region_df = pd.read_csv('geoMap.csv')
search_trends_by_region_df.head()

,,,,,Category: Music & Audio
Country,Ableton Live: (1/1/04 - 10/24/25),Cubase: (1/1/04 - 10/24/25),FL Studio: (1/1/04 - 10/24/25),Pro Tools: (1/1/04 - 10/24/25),Logic Pro: (1/1/04 - 10/24/25)
Solomon Islands,NaN,NaN,NaN,NaN,NaN
Iceland,NaN,NaN,NaN,NaN,NaN
British Virgin Islands,NaN,NaN,NaN,NaN,NaN
Faroe Islands,NaN,NaN,NaN,NaN,NaN


In [19]:
search_trends_by_region_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 251 entries, ('Country', 'Ableton Live: (1/1/04 - 10/24/25)', 'Cubase: (1/1/04 - 10/24/25)', 'FL Studio: (1/1/04 - 10/24/25)', 'Pro Tools: (1/1/04 - 10/24/25)') to ('South Sudan', nan, nan, nan, nan)
Data columns (total 1 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Category: Music & Audio  59 non-null     object
dtypes: object(1)
memory usage: 17.7+ KB


It looks like only 59 regions have usable data, so I'll drop the ones that don't, and rename any remaining NaN values to show lack of interest.

In [20]:
search_trends_by_region_df.dropna(inplace=True)
search_trends_by_region_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 59 entries, ('Country', 'Ableton Live: (1/1/04 - 10/24/25)', 'Cubase: (1/1/04 - 10/24/25)', 'FL Studio: (1/1/04 - 10/24/25)', 'Pro Tools: (1/1/04 - 10/24/25)') to ('Iran', '5%', '13%', '79%', '2%')
Data columns (total 1 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Category: Music & Audio  59 non-null     object
dtypes: object(1)
memory usage: 15.0+ KB


In [21]:
search_trends_by_region_df.head()

,,,,,Category: Music & Audio
Country,Ableton Live: (1/1/04 - 10/24/25),Cubase: (1/1/04 - 10/24/25),FL Studio: (1/1/04 - 10/24/25),Pro Tools: (1/1/04 - 10/24/25),Logic Pro: (1/1/04 - 10/24/25)
Australia,35%,10%,24%,23%,8%
Switzerland,34%,17%,28%,12%,9%
Belgium,32%,13%,37%,12%,6%
Netherlands,26%,14%,44%,10%,6%


In [22]:
search_trends_by_region_df.isna()

,,,,,Category: Music & Audio
Country,Ableton Live: (1/1/04 - 10/24/25),Cubase: (1/1/04 - 10/24/25),FL Studio: (1/1/04 - 10/24/25),Pro Tools: (1/1/04 - 10/24/25),False
Australia,35%,10%,24%,23%,False
Switzerland,34%,17%,28%,12%,False
Belgium,32%,13%,37%,12%,False
Netherlands,26%,14%,44%,10%,False
Norway,22%,15%,39%,16%,False
Estonia,22%,11%,55%,8%,False
Ireland,35%,9%,25%,24%,False
Denmark,28%,13%,35%,15%,False
United Kingdom,31%,17%,23%,17%,False


No NaN values seem to remain, so I think I'm calling the data __successfully cleaned!__